In [1]:
import pixiedust
#!pip install --upgrade ~/Development/repos/quantutils
#!pip install --upgrade ~/Development/repos/tradeframework


# import utilities
from MIPriceStore.api.datasource import MIDataStore
from quantutils.api.auth import CredentialsStore
import quantutils.dataset.pipeline as ppl

#import tradeframework
from tradeframework.api import Derivative, Optimizer, Model, TradeEnvironment, TradeEngine, Asset
from tradeframework.environments import SandboxEnvironment, ZiplineEnvironment
from tradeframework.models import PreOpenMomentum
import tradeframework.operations.trader as trader
import tradeframework.operations.utils as utils

# Development
import pandas
import numpy as np


%matplotlib inline

Pixiedust database opened successfully


Pixiedust runtime updated. Please restart kernel


In [4]:
## Get Market Data

#mds = MIDataStore(remote=True, location="http://pricestore.192.168.1.203.nip.io")
mds = MIDataStore(location="../marketinsights-price-store/datastore")

marketData = mds.aggregate("2013-01-01", "2013-07-11", ["WallSt-hourly","D%26J-IND"],"H")
ts = ppl.removeNaNs(marketData)
ts.index = ts.index.tz_localize('UTC')

sample1 = ts[5:10].copy()
sample2 = ts[12:15].copy()
sample3 = ts[:5].copy()

Loading data from WallSt-hourly in ../marketinsights-price-store/datastore/data.hdf


AttributeError: 'numpy.bytes_' object has no attribute 'get'

In [16]:
sample3

,Open,High,Low,Close
2013-01-02 15:00:00+00:00,13366.48,13374.82,13338.68,13345.45
2013-01-02 16:00:00+00:00,13345.60,13353.75,13321.64,13321.77
2013-01-02 17:00:00+00:00,13321.76,13339.25,13320.28,13326.66
2013-01-02 18:00:00+00:00,13326.64,13336.06,13322.51,13329.19
2013-01-02 19:00:00+00:00,13329.09,13339.82,13318.19,13331.80


In [11]:
sample1

,Open,High,Low,Close
Date_Time,,,,
2013-01-02 20:00:00+00:00,13331.83,13395.20,13331.83,13395.20
2013-01-03 15:00:00+00:00,13381.66,13395.66,13376.42,13391.14
2013-01-03 16:00:00+00:00,13391.18,13413.73,13391.16,13408.19
2013-01-03 17:00:00+00:00,13408.16,13420.65,13402.45,13419.40
2013-01-03 18:00:00+00:00,13419.37,13430.13,13411.07,13427.51


In [15]:
sample2

,Open,High,Low,Close
Date_Time,,,,
2013-01-04 15:00:00+00:00,13407.46,13411.91,13377.22,13387.45
2013-01-04 16:00:00+00:00,13387.37,13420.86,13386.80,13408.33
2013-01-04 17:00:00+00:00,13408.20,13415.80,13403.52,13405.06


In [16]:
sample3

,Open,High,Low,Close
Date_Time,,,,
2013-01-02 15:00:00+00:00,13366.48,13374.82,13338.68,13345.45
2013-01-02 16:00:00+00:00,13345.60,13353.75,13321.64,13321.77
2013-01-02 17:00:00+00:00,13321.76,13339.25,13320.28,13326.66
2013-01-02 18:00:00+00:00,13326.64,13336.06,13322.51,13329.19
2013-01-02 19:00:00+00:00,13329.09,13339.82,13318.19,13331.80


In [24]:
def appendHDF(hdfFile, bucket, data, sample_unit='1H', update=False):
    # Get HDFStore
    hdfStore = pandas.HDFStore(hdfFile, 'a')
    append = True
    # TODO Sort incoming data
    
    if '/'+bucket in hdfStore.keys():
    
        # Get first,last row
        nrows = hdfStore.get_storer(bucket).nrows
        last = hdfStore.select(bucket,start=nrows-1,stop=nrows)
        
        # If this is entirely beyond the last element in the file... append
        # If not... update (incurring a full file re-write and performance hit), or throw exception                  
        if not data[data.index<=last.index[0]].empty:
            # Update table with overlapped data
            storedData = hdfStore.get(bucket)
            data = ppl.merge(data, storedData)
            append = False
            
            if not update:
                raise ValueError('Error: Entry already exists for data starting at index ' + str(data.index[0])) 
        else:                                      
            data = ppl.merge(last, data)        
     
    data = ppl.resample(data, sample_unit)
    if append:        
        print("Appending data...")
        hdfStore.append(bucket, data, format='table', append=True)
    else:
        print("Re-writing table data for update...")
        hdfStore.put(bucket, data, format='table')

    hdfStore.close()
    
    
def getHDF(hdfFile, bucket):
    
    # Get HDFStore
    hdfStore = pandas.HDFStore(hdfFile, 'r')
    data = hdfStore.get(bucket)   
    hdfStore.close()    
    return data



In [27]:
appendHDF("temp.hdf", "DOW", sample3, '1H', True)

Merging data...
Resampling to 1H periods
Re-writing table data for update...


In [15]:
hdfFile = "tmp/data.hdf"
x = pandas.HDFStore(hdfFile, 'r')

In [20]:
start = "2013-01-02 20:00:00+00:00"
end = "2013-01-03 17:00:00+00:00"
select = ''.join(["index>'", start, "' and index<='", end, "'"])
poo = x.select("D&J-IND", where=select)

In [18]:
x.keys()

['/D&J-IND', '/SANDP-500', '/SP500-hourly', '/WallSt-hourly']

In [21]:
poo

,Open,High,Low,Close
Date_Time,,,,
2013-01-02 20:05:00+00:00,13338.40,13342.17,13338.40,13342.17
2013-01-02 20:10:00+00:00,13342.19,13342.50,13339.98,13342.45
2013-01-02 20:15:00+00:00,13342.47,13344.33,13340.62,13344.33
2013-01-02 20:20:00+00:00,13344.40,13353.31,13344.40,13350.49
2013-01-02 20:25:00+00:00,13350.19,13350.78,13343.62,13347.16
...,...,...,...,...
2013-01-03 16:40:00+00:00,13410.17,13410.54,13404.51,13404.51
2013-01-03 16:45:00+00:00,13404.50,13409.56,13404.12,13409.55
2013-01-03 16:50:00+00:00,13409.56,13412.46,13408.55,13409.63
